In [ ]:
import os
import shutil
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import random
from google.colab import drive

# --- 1. MOUNT DRIVE ---
drive.mount('/content/drive')

# --- 2. RESTORE MODEL CODE ---
if not os.path.exists('/content/models'):
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')
    print("Models restored.")

# --- 3. RESTORE DATASET ---
# Check if dataset exists, if not, unzip it
if not os.path.exists('/content/dataset/project_data/gta5'):
    print("Dataset not found. checking for zip file...")

    # Check paths (semseg folder first, then root)
    zip_path_1 = '/content/drive/MyDrive/semseg/project_data.zip'
    zip_path_2 = '/content/drive/MyDrive/project_data.zip'

    if os.path.exists(zip_path_1):
        print(f"Unzipping from {zip_path_1}...")
        shutil.unpack_archive(zip_path_1, '/content/dataset')
        print("Dataset extracted!")
    elif os.path.exists(zip_path_2):
        print(f"Unzipping from {zip_path_2}...")
        shutil.unpack_archive(zip_path_2, '/content/dataset')
        print("Dataset extracted!")
    else:
        print("Error: 'project_data.zip' not found in Drive. Please check your path.")
else:
    print("Dataset is ready.")

# Add models to path
sys.path.append('/content/models')

Mounted at /content/drive
Cloning into 'temp_repo'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
Receiving objects: 100% (34/34), 11.29 KiB | 11.29 MiB/s, done.
Resolving deltas: 100% (9/9), done.
remote: Total 34 (delta 9), reused 3 (delta 3), pack-reused 13 (from 1)
Models restored.
Dataset not found. checking for zip file...
Unzipping from /content/drive/MyDrive/semseg/project_data.zip...
Dataset extracted!


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
import random
import torch.nn.functional as F
from models.bisenet.build_bisenet import BiSeNet

# --- CONFIGURATION ---

CHECKPOINT_NAME = 'bisenet_dacs_thresholded.pth'
EPOCHS = 50
BATCH_SIZE = 8
LR = 2.5e-2
CONFIDENCE_THRESHOLD = 0.968
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
GTA_PATH = '/content/dataset/project_data/gta5'
CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'
SAVE_PATH = f'/content/drive/MyDrive/semseg/{CHECKPOINT_NAME}'

# --- DATASET CLASS ---
class GTA5_City_Dataset(Dataset):
    def __init__(self, gta_root, city_root):
        self.gta_images_dir = os.path.join(gta_root, 'images')
        self.gta_masks_dir = os.path.join(gta_root, 'labels')
        self.gta_images = sorted(os.listdir(self.gta_images_dir))

        self.city_images_dir = os.path.join(city_root, 'leftImg8bit', 'train')
        self.city_images = []
        if os.path.exists(self.city_images_dir):
            for city in os.listdir(self.city_images_dir):
                c_path = os.path.join(self.city_images_dir, city)
                if os.path.isdir(c_path):
                    for f in os.listdir(c_path):
                        if f.endswith('_leftImg8bit.png'):
                            self.city_images.append(os.path.join(c_path, f))

        # Color Jitter for Source (GTA) - Essential for Domain Adaptation
        self.color_jitter = transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)
        self.normalize = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }
        self.to_tensor = transforms.ToTensor()

    def __len__(self): return len(self.gta_images)

    def __getitem__(self, idx):
        # Load Source (GTA)
        gta_path = os.path.join(self.gta_images_dir, self.gta_images[idx])
        mask_path = os.path.join(self.gta_masks_dir, self.gta_images[idx])

        gta_img = Image.open(gta_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        gta_mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        # Load Target (Cityscapes)
        rand_idx = random.randint(0, len(self.city_images) - 1)
        city_img = Image.open(self.city_images[rand_idx]).convert('RGB').resize((1280, 720), Image.BILINEAR)

        # Apply Transforms
        gta_img_aug = self.color_jitter(gta_img)
        gta_t = self.normalize(self.to_tensor(gta_img_aug))
        city_t = self.normalize(self.to_tensor(city_img))

        # Process Mask
        mask_np = np.array(gta_mask)
        gta_lbl = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): gta_lbl[mask_np == k] = v

        return gta_t, torch.from_numpy(gta_lbl).long(), city_t

# --- DACS MIXING FUNCTION ---
def dacs_mix(gta_img, gta_lbl, city_img, city_pseudo_lbl):
    """
    Mixes Source (GTA) classes onto Target (Cityscapes) images.
    """
    mixed_img = city_img.clone()
    mixed_lbl = city_pseudo_lbl.clone()

    for i in range(gta_img.shape[0]):
        classes_in_img = torch.unique(gta_lbl[i])
        classes_in_img = classes_in_img[classes_in_img != 255]

        if len(classes_in_img) > 0:
            n_classes = len(classes_in_img)

            perm = torch.randperm(n_classes)
            selected_classes = classes_in_img[perm[:(n_classes + 1) // 2]]

            mask = torch.zeros_like(gta_lbl[i]).bool()
            for c in selected_classes:
                mask = mask | (gta_lbl[i] == c)

            mixed_img[i, :, mask] = gta_img[i, :, mask]
            mixed_lbl[i, mask] = gta_lbl[i, mask]

    return mixed_img, mixed_lbl

# --- TRAINING SETUP ---
print("Starting Advanced DACS Training (With Thresholding)...")

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

# Poly Scheduler
scheduler = optim.lr_scheduler.PolyLR(optimizer, total_iters=EPOCHS, power=0.9) if hasattr(optim.lr_scheduler, 'PolyLR') else optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# Loss Function
criterion = nn.CrossEntropyLoss(ignore_index=255)

dataset = GTA5_City_Dataset(GTA_PATH, CITYSCAPES_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

if os.path.exists(SAVE_PATH):
    print("Resuming from checkpoint...")
    checkpoint = torch.load(SAVE_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint.get('epoch', 0) + 1
else:
    start_epoch = 0

# --- TRAINING LOOP ---
for epoch in range(start_epoch, EPOCHS):
    model.train()
    for i, (gta_img, gta_lbl, city_img) in enumerate(loader):
        gta_img, gta_lbl, city_img = gta_img.to(DEVICE), gta_lbl.to(DEVICE), city_img.to(DEVICE)

        optimizer.zero_grad()

        # 1. Generate Pseudo-Labels for Cityscapes
        with torch.no_grad():
            city_out = model(city_img)
            if isinstance(city_out, tuple): city_out = city_out[0]

            probs = torch.softmax(city_out, dim=1)
            max_probs, city_pseudo_lbl = torch.max(probs, dim=1)

            # --- CRITICAL FIX: THRESHOLDING ---

            city_pseudo_lbl[max_probs < CONFIDENCE_THRESHOLD] = 255

        # 2. Mix Images (ClassMix)
        mixed_img, mixed_lbl = dacs_mix(gta_img, gta_lbl, city_img, city_pseudo_lbl)

        # 3. Train on Mixed Images
        out = model(mixed_img)

        # Loss calculation (Standard BiSeNet Multi-head Loss)
        loss = criterion(out[0], mixed_lbl)
        loss += 0.1 * criterion(out[1], mixed_lbl) + 0.1 * criterion(out[2], mixed_lbl)

        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}] Step [{i}/{len(loader)}] Loss: {loss.item():.4f}")

    if hasattr(optimizer, 'get_last_lr'): scheduler.step()

    torch.save({'model_state_dict': model.state_dict(), 'epoch': epoch}, SAVE_PATH)
    print(f"Epoch {epoch+1} Saved.")

Starting Advanced DACS Training (With Thresholding)...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 191MB/s]


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:01<00:00, 175MB/s]


Epoch [1/50] Step [0/313] Loss: 4.7641
Epoch [1/50] Step [50/313] Loss: 0.7572
Epoch [1/50] Step [100/313] Loss: 0.5669
Epoch [1/50] Step [150/313] Loss: 0.4181
Epoch [1/50] Step [200/313] Loss: 0.4624
Epoch [1/50] Step [250/313] Loss: 0.4083
Epoch [1/50] Step [300/313] Loss: 0.3949
Epoch 1 Saved.
Epoch [2/50] Step [0/313] Loss: 0.5912
Epoch [2/50] Step [50/313] Loss: 0.3660
Epoch [2/50] Step [100/313] Loss: 0.3352
Epoch [2/50] Step [150/313] Loss: 0.3606
Epoch [2/50] Step [200/313] Loss: 0.3330
Epoch [2/50] Step [250/313] Loss: 0.3450
Epoch [2/50] Step [300/313] Loss: 0.3020
Epoch 2 Saved.
Epoch [3/50] Step [0/313] Loss: 0.3918
Epoch [3/50] Step [50/313] Loss: 0.3339
Epoch [3/50] Step [100/313] Loss: 0.3510
Epoch [3/50] Step [150/313] Loss: 0.3672
Epoch [3/50] Step [200/313] Loss: 0.4017
Epoch [3/50] Step [250/313] Loss: 0.2865
Epoch [3/50] Step [300/313] Loss: 0.2354
Epoch 3 Saved.
Epoch [4/50] Step [0/313] Loss: 0.2596
Epoch [4/50] Step [50/313] Loss: 0.3310
Epoch [4/50] Step [100/3

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
import random
from models.bisenet.build_bisenet import BiSeNet

# Configuration
CHECKPOINT_NAME = 'bisenet_dacs_checkpoint.pth'
EPOCHS = 50
BATCH_SIZE = 8
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
GTA_PATH = '/content/dataset/project_data/gta5'
CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'
SAVE_PATH = f'/content/drive/MyDrive/semseg/{CHECKPOINT_NAME}'

class GTA5_City_Dataset(Dataset):
    def __init__(self, gta_root, city_root):
        self.gta_images_dir = os.path.join(gta_root, 'images')
        self.gta_masks_dir = os.path.join(gta_root, 'labels')
        self.gta_images = sorted(os.listdir(self.gta_images_dir))

        self.city_images_dir = os.path.join(city_root, 'leftImg8bit', 'train')
        self.city_images = []
        if os.path.exists(self.city_images_dir):
            for city in os.listdir(self.city_images_dir):
                c_path = os.path.join(self.city_images_dir, city)
                if os.path.isdir(c_path):
                    for f in os.listdir(c_path):
                        if f.endswith('_leftImg8bit.png'):
                            self.city_images.append(os.path.join(c_path, f))

        self.normalize = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }

        self.to_tensor = transforms.ToTensor()

    def __len__(self): return len(self.gta_images)

    def __getitem__(self, idx):
        gta_path = os.path.join(self.gta_images_dir, self.gta_images[idx])
        mask_path = os.path.join(self.gta_masks_dir, self.gta_images[idx])

        gta_img = Image.open(gta_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        gta_mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        rand_idx = random.randint(0, len(self.city_images) - 1)
        city_img = Image.open(self.city_images[rand_idx]).convert('RGB').resize((1280, 720), Image.BILINEAR)

        gta_t = self.normalize(self.to_tensor(gta_img))
        city_t = self.normalize(self.to_tensor(city_img))

        mask_np = np.array(gta_mask)
        gta_lbl = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): gta_lbl[mask_np == k] = v

        return gta_t, torch.from_numpy(gta_lbl).long(), city_t

def dacs_mix(gta_img, gta_lbl, city_img, city_pseudo_lbl):
    batch_size = gta_img.shape[0]
    mixed_img = city_img.clone()
    mixed_lbl = city_pseudo_lbl.clone()

    for i in range(batch_size):
        classes_in_img = torch.unique(gta_lbl[i])
        classes_in_img = classes_in_img[classes_in_img != 255]

        if len(classes_in_img) > 0:
            n_classes = len(classes_in_img)
            perm = torch.randperm(n_classes)
            selected_classes = classes_in_img[perm[:(n_classes + 1) // 2]]

            mask = torch.zeros_like(gta_lbl[i]).bool()
            for c in selected_classes:
                mask = mask | (gta_lbl[i] == c)

            mixed_img[i, :, mask] = gta_img[i, :, mask]
            mixed_lbl[i, mask] = gta_lbl[i, mask]

    return mixed_img, mixed_lbl

print("Resuming DACS Training...")

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=255)

dataset = GTA5_City_Dataset(GTA_PATH, CITYSCAPES_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

if os.path.exists(SAVE_PATH):
    print("Found checkpoint. Loading...")
    checkpoint = torch.load(SAVE_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint.get('epoch', 0) + 1
    print(f"Starting from Epoch {start_epoch}")
else:
    print("No checkpoint found. Starting from Epoch 0")
    start_epoch = 0

for epoch in range(start_epoch, EPOCHS):
    model.train()
    for i, (gta_img, gta_lbl, city_img) in enumerate(loader):
        gta_img = gta_img.to(DEVICE)
        gta_lbl = gta_lbl.to(DEVICE)
        city_img = city_img.to(DEVICE)

        optimizer.zero_grad()

        with torch.no_grad():
            city_out = model(city_img)
            if isinstance(city_out, tuple): city_out = city_out[0]
            city_pseudo_lbl = torch.argmax(city_out, dim=1)

        mixed_img, mixed_lbl = dacs_mix(gta_img, gta_lbl, city_img, city_pseudo_lbl)

        out = model(mixed_img)
        loss = criterion(out[0], mixed_lbl) + 0.1 * criterion(out[1], mixed_lbl) + 0.1 * criterion(out[2], mixed_lbl)

        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}] Step [{i}/{len(loader)}] Loss: {loss.item():.4f}")

    torch.save({'model_state_dict': model.state_dict(), 'epoch': epoch}, SAVE_PATH)
    print(f"Epoch {epoch+1} Saved.")

Resuming DACS Training...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 164MB/s]


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:00<00:00, 182MB/s]


Found checkpoint. Loading...
Starting from Epoch 28
Epoch [29/50] Step [0/313] Loss: 0.3165
Epoch [29/50] Step [50/313] Loss: 0.2518
Epoch [29/50] Step [100/313] Loss: 0.2992
Epoch [29/50] Step [150/313] Loss: 0.2459
Epoch [29/50] Step [200/313] Loss: 0.4181
Epoch [29/50] Step [250/313] Loss: 0.2636
Epoch [29/50] Step [300/313] Loss: 0.3092
Epoch 29 Saved.
Epoch [30/50] Step [0/313] Loss: 0.2924
Epoch [30/50] Step [50/313] Loss: 0.2786
Epoch [30/50] Step [100/313] Loss: 0.2836
Epoch [30/50] Step [150/313] Loss: 0.2963
Epoch [30/50] Step [200/313] Loss: 0.3433
Epoch [30/50] Step [250/313] Loss: 0.3134
Epoch [30/50] Step [300/313] Loss: 0.3390
Epoch 30 Saved.
Epoch [31/50] Step [0/313] Loss: 0.2895
Epoch [31/50] Step [50/313] Loss: 0.2003
Epoch [31/50] Step [100/313] Loss: 0.3458
Epoch [31/50] Step [150/313] Loss: 0.2423
Epoch [31/50] Step [200/313] Loss: 0.2484
Epoch [31/50] Step [250/313] Loss: 0.4212
Epoch [31/50] Step [300/313] Loss: 0.3924
Epoch 31 Saved.
Epoch [32/50] Step [0/313] 

In [ ]:
import torch
import numpy as np
import os
import sys
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms

# --- PATH SETUP ---
if os.path.exists("/content/MLDL2024_project1"):
    sys.path.append("/content/MLDL2024_project1")
from models.bisenet.build_bisenet import BiSeNet

# --- CONFIG ---
# POINTING TO THE CORRECT FILE (The one you trained for 2 days)
CHECKPOINT_PATH = "/content/drive/MyDrive/semseg/bisenet_dacs_thresholded.pth"
CITY_PATH = "/content/dataset/project_data/cityscapes"
NUM_CLASSES = 19
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- DATASET ---
class CityscapesValDataset(Dataset):
    def __init__(self, root):
        self.img_dir = os.path.join(root, "leftImg8bit", "val")
        self.lbl_dir = os.path.join(root, "gtFine", "val")
        self.imgs = []
        self.lbls = []

        if os.path.exists(self.img_dir):
            for city in sorted(os.listdir(self.img_dir)):
                img_path = os.path.join(self.img_dir, city)
                lbl_path = os.path.join(self.lbl_dir, city)
                for f in sorted(os.listdir(img_path)):
                    if f.endswith("_leftImg8bit.png"):
                        self.imgs.append(os.path.join(img_path, f))
                        self.lbls.append(os.path.join(lbl_path, f.replace("_leftImg8bit.png", "_gtFine_labelTrainIds.png")))

        self.transform = transforms.Compose([
            transforms.Resize((512, 1024)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self): return len(self.imgs)

    def __getitem__(self, idx):
        img = Image.open(self.imgs[idx]).convert("RGB")
        lbl = Image.open(self.lbls[idx]).resize((1024, 512), Image.NEAREST)
        img = self.transform(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        return img, lbl

# --- EVALUATION ---
print(f"Loading Checkpoint: {CHECKPOINT_PATH}")
model = BiSeNet(num_classes=NUM_CLASSES, context_path="resnet18").to(DEVICE)

if os.path.exists(CHECKPOINT_PATH):
    ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    # The 'thresholded' script saved simple 'model_state_dict'
    if 'model_state_dict' in ckpt:
        model.load_state_dict(ckpt['model_state_dict'])
        print("✅ SUCCESS: Model weights loaded.")
    else:
        model.load_state_dict(ckpt)
        print("Model weights loaded (Direct).")
else:
    print(f"Error: {CHECKPOINT_PATH} not found.")
    sys.exit()

model.eval()
loader = DataLoader(CityscapesValDataset(CITY_PATH), batch_size=1, shuffle=False, num_workers=2)

hist = np.zeros((NUM_CLASSES, NUM_CLASSES))
print("Starting Evaluation...")

with torch.no_grad():
    for img, lbl in tqdm(loader):
        img = img.to(DEVICE)

        # Robust inference
        output = model(img)
        if isinstance(output, (list, tuple)):
            output = output[0]

        preds = torch.argmax(output, dim=1).cpu().numpy()
        lbl = lbl.numpy()

        if preds.ndim == 3: preds = preds[0]
        if lbl.ndim == 3: lbl = lbl[0]

        mask = (lbl >= 0) & (lbl < NUM_CLASSES)
        hist += np.bincount(
            NUM_CLASSES * lbl[mask].astype(int) + preds[mask],
            minlength=NUM_CLASSES ** 2
        ).reshape(NUM_CLASSES, NUM_CLASSES)

iou = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
miou = np.nanmean(iou) * 100

print("\n" + "="*30)
print(f"Final DACS (Thresholded) mIoU: {miou:.2f}%")
print("="*30)

CLASSES = [
    "Road", "Sidewalk", "Building", "Wall", "Fence", "Pole",
    "Traffic Light", "Traffic Sign", "Vegetation", "Terrain", "Sky",
    "Person", "Rider", "Car", "Truck", "Bus", "Train", "Motorcycle", "Bicycle"
]
print("-" * 30)
for i, name in enumerate(CLASSES):
    print(f"{name:15s}: {iou[i]*100:.2f}%")
print("-" * 30)

Loading Checkpoint: /content/drive/MyDrive/semseg/bisenet_dacs_thresholded.pth
✅ SUCCESS: Model weights loaded.
Starting Evaluation...


100%|██████████| 500/500 [00:56<00:00,  8.80it/s]


Final DACS (Thresholded) mIoU: 24.83%
------------------------------
Road           : 86.92%
Sidewalk       : 16.52%
Building       : 57.81%
Wall           : 7.16%
Fence          : 11.77%
Pole           : 16.09%
Traffic Light  : 8.06%
Traffic Sign   : 6.84%
Vegetation     : 66.26%
Terrain        : 12.73%
Sky            : 50.30%
Person         : 33.31%
Rider          : 3.52%
Car            : 72.31%
Truck          : 11.48%
Bus            : 4.44%
Train          : 0.00%
Motorcycle     : 6.23%
Bicycle        : 0.00%
------------------------------
